# Определение стоимости автомобилей

**Описание проекта**

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля.
Постройте модель, которая умеет её определять. В вашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.

Критерии, которые важны заказчику:
- качество предсказания;
- время обучения модели;
- время предсказания модели.

**Описание данных** 
Данные находятся в файле /datasets/autos.csv. Скачать датасет.

Признаки

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак

- Price — цена (евро)

**План работы**

[1  Подготовка данных](#section1)

[1.1  Изучение данных](#section1.1)

[2  Предобработка данных](#section1.2)

- [2.1  Обработка пропусков](#section2.1)
- [2.2  Подготовка выборок](#section2.2)
- [2.3  Кодирование и масштабирование](#section2.3)

[2  Обучение моделей](#section3)

- [2.1  DummyRegressor](#section3.1)
- [2.2  LinearRegression](#section3.2)
- [2.3  CatBoostRegressor](#section3.3)
- [2.4  LGBMRegressor](#section3.4)

[4  Анализ моделей](#section4)

[4.1 Тестирование лучших моделей](#section4.1)

[5  Общий вывод](#section5)

<a id='section1'></a>
## Подготовка данных

<a id='section1.1'></a>
### Изучение данных

In [1]:
# Загружаем необходимы библиотеки для работы
import pandas as pd
import numpy as np
import re
from scipy import stats as st
from IPython.display import display
from numpy.random import RandomState
from scipy import stats

from catboost import CatBoostRegressor, cv, Pool, train
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error as mse

import lightgbm
import catboost
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# сохраним датафрейм в переменную
df = pd.read_csv('/datasets/autos.csv')

Выведем таблицу, изучим структуры таблиц: типы данных, количество строк, столбцов, пропущенных данных.

In [3]:
display(df)
print(df.info())
print(df.describe())
print(df.nunique())
print('Количество дубликатов:', sum(df.duplicated()))

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

**Вывод:**
- В наборе данных 350000+ строк .
- Названия столбцов стоит немного изменить, чтобы было удобнее к ним обращаться.
- Данные содержат пропуски.
- Данные имеют тип object - категории имеет смысл закодировать, особенно, если их немного.

head():
- Имеются нули и NaN. Стоит обратить внимание и возможно избавиться от таковых.
- petrol и gasoline - наименование одного и того же топлива, стоит приравнять.
- DateCrawled повторяет дату в признаке DateCreated.
- Месяц регистрации может не имеет значения, в отличае от года.
- Почтовый индекс вряд ли нужен.

describe():
- Имеются выбросы - нулевые значения месяца, мощности и стоимости.
- 1000 год и 9999. Возможно, это ошибка занесения данных в базу или выгрузки. Можно удалить такие данные, тем более, что 1 и 3 квантили на месте.
- Мощность 20000 л.с.

nunique():
- Фотографий нет, столбец на удаление.
- 13 месяцев - один лишний. Да и вряд ли они вообще понадобятся.
- 40 марок автомобилей и 250 моделей.
- 151 вариант года регистрации.
- Данные о ремонте - 2 значения: 'yes' и 'no'. NaN скорее означает отсутствие записей.
- Пробег явно имеет округлённое значение.
- Индексов очень много.

<a id='section2'></a>
## Предобработка данных

In [4]:
# Переведём все названия столбцов в нижний регистр
df.columns = df.columns.str.lower()
df.columns

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'kilometer', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [5]:
# удалим лишние столбцы, оставив индекс для поиска дубликатов далее
df.drop(['datecrawled', 'numberofpictures', 'registrationmonth', 'lastseen'], axis=1, inplace=True)

In [6]:
# Приведём оставшиеся столбцы к более сокращенному виду для удобства восприятия таблицы
df.rename({'vehicletype' : 'body',
             'registrationyear': 'year_reg',
             'kilometer': 'mileage',
             'gearbox': 'gear',
             'datecreated': 'date',
             'notrepaired': 'repair',
             'fueltype': 'fuel'}, axis = 1, inplace = True)

In [7]:
df.head()

,price,body,year_reg,gear,power,model,mileage,fuel,brand,repair,date,postalcode
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN,2016-03-24 00:00:00,70435
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes,2016-03-24 00:00:00,66954
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN,2016-03-14 00:00:00,90480
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,2016-03-17 00:00:00,91074
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,2016-03-31 00:00:00,60437


<a id='section2.1'></a>
### Обработка пропусков

In [8]:
# удалим дубликаты
df = df.drop_duplicates()

Признак **price**

In [9]:
f"Количество объявлений с нулевой ценой: {len(df.loc[df['price'] == 0])}"

'Количество объявлений с нулевой ценой: 10521'

Данные о "нулевой" стоимости явно ошибка (или цена не была указана продавцом), возможности востановить нет, а так как это целевой признак в этих объектах, удалим данные строки.

In [10]:
df = df.loc[df['price'] != 0]

Признак **model**

In [11]:
f"Количество объявлений с незаполненной моделью: {len(df.loc[df['model'].isna()])}"

'Количество объявлений с незаполненной моделью: 17273'

Данный признак является одним из основных для предсказания цены. Возможности восстановить корректно эти данные нет, удалим пропуски.

In [12]:
df = df.loc[~df['model'].isna()]

Признак **power**

In [13]:
f"Количество автомобилей с мощностью > 1000 л.с.: {len(df.loc[(df['power'] > 1000) | (df['power'] <= 0)])}"

'Количество автомобилей с мощностью > 1000 л.с.: 30889'

Мощность > 1000 л.с. уже излишняя. Возможно допустили ошибку при вводе данных, где по умолчанию стоял "0" и он попал в конец итогового значения, точно не известно. Заполним такие значения медианными среди автомобилей той же модели.

In [14]:
df.loc[(df['power'] > 1000) | (df['power'] <= 0), 'power'] = None
df['power'] = df['power'].fillna(df.groupby('model')['power'].transform('median'))
df = df.loc[~df['power'].isna()]
df['power'] = df['power'].astype('int64')

Признак **year_reg**

In [15]:
f"Количество объявлений с неправильным годом регистрации: {len(df.loc[df['year_reg'] > 2022])}"

'Количество объявлений с неправильным годом регистрации: 56'

In [16]:
df.loc[df['year_reg'] > 2022].head(10)

,price,body,year_reg,gear,power,model,mileage,fuel,brand,repair,date,postalcode
12946,49,NaN,5000,NaN,101,golf,5000,NaN,volkswagen,NaN,2016-03-29 00:00:00,74523
17271,700,NaN,9999,NaN,116,other,10000,NaN,opel,NaN,2016-03-23 00:00:00,21769
18259,300,NaN,2200,NaN,58,twingo,5000,NaN,renault,NaN,2016-03-16 00:00:00,45307
26382,150,NaN,3000,NaN,95,a_klasse,5000,NaN,mercedes_benz,NaN,2016-03-07 00:00:00,28217
26560,800,NaN,2066,NaN,125,zafira,5000,gasoline,opel,NaN,2016-03-24 00:00:00,45355
28390,799,NaN,9999,NaN,150,3er,10000,petrol,bmw,NaN,2016-04-05 00:00:00,72116
28965,18000,NaN,9999,NaN,95,a_klasse,10000,petrol,mercedes_benz,NaN,2016-04-04 00:00:00,51379
29426,7999,NaN,9999,NaN,44,kaefer,10000,NaN,volkswagen,NaN,2016-03-23 00:00:00,47638
40954,59,NaN,8000,NaN,116,other,10000,NaN,chevrolet,NaN,2016-03-07 00:00:00,31061
45662,999,NaN,9000,NaN,69,fiesta,10000,NaN,ford,yes,2016-03-23 00:00:00,50733


Предсказать цену с таким годом будет сложно предсказать, и подобные объявления имеют так же много пропусков и могут внести ошибку в модель. Так как их достаточно мало и они имеют много пропусков - удалим такие объявления.

In [17]:
df = df.loc[df['year_reg'] <= 2016]

Признаки **body**, **gear**, **fuel**

In [18]:
# Переименуем gasoline на petrol, так как это один тип топлива
df.fuel.where(df.fuel != 'gasoline', 'petrol', inplace=True)

Данные параметры c большой вероятностью одинаковы в одинаковых моделях автомобилей. Заполним пропуски в данных признаках наиболее частым значением той же модели.

In [19]:
df['body'] = df['body'].fillna(df
                                                  .groupby('model')['body']
                                                  .transform(lambda x: x.value_counts().idxmax())
                                                 )

In [20]:
df['gear'] = df['gear'].fillna(df
                                         .groupby('model')['gear']
                                         .transform(lambda x: x.value_counts().idxmax())
                                        )

In [21]:
df['fuel'] = df['fuel'].fillna(df
                                           .groupby('model')['fuel']
                                           .transform(lambda x: x.value_counts().idxmax())
                                          )

Признак **repair**

In [22]:
# закроим пропуски "заглушкой"
df['repair'] = df['repair'].fillna('unknown')

Признаки **date**

Извлечём из неё год и месяц создания объявления. Заодно извлечём день, чтобы в дальнейшем проверить дубликаты с точностью до дня.

In [23]:
# Изменим тип данных
df.date = pd.to_datetime(df.date)

In [24]:
# Создадим столбцы "год" и "месяц"
df.loc[:, 'year_created'] = df.date.dt.year
df.loc[:, 'month_created'] = df.date.dt.month
df.loc[:, 'day_created'] = df.date.dt.day

In [25]:
df.drop('date', axis=1, inplace=True)

In [26]:
df.head(10)

,price,body,year_reg,gear,power,model,mileage,fuel,brand,repair,postalcode,year_created,month_created,day_created
0,480,sedan,1993,manual,101,golf,150000,petrol,volkswagen,unknown,70435,2016,3,24
2,9800,suv,2004,auto,163,grand,125000,petrol,jeep,unknown,90480,2016,3,14
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,91074,2016,3,17
4,3600,small,2008,manual,69,fabia,90000,petrol,skoda,no,60437,2016,3,31
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,33775,2016,4,4
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no,67112,2016,4,1
8,14500,bus,2014,manual,125,c_max,30000,petrol,ford,unknown,94505,2016,4,4
9,999,small,1998,manual,101,golf,150000,petrol,volkswagen,unknown,27472,2016,3,17
10,2000,sedan,2004,manual,105,3_reihe,150000,petrol,mazda,no,96224,2016,3,26
11,2799,wagon,2005,manual,140,passat,150000,petrol,volkswagen,yes,57290,2016,4,7


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304076 entries, 0 to 354368
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   price          304076 non-null  int64 
 1   body           304076 non-null  object
 2   year_reg       304076 non-null  int64 
 3   gear           304076 non-null  object
 4   power          304076 non-null  int64 
 5   model          304076 non-null  object
 6   mileage        304076 non-null  int64 
 7   fuel           304076 non-null  object
 8   brand          304076 non-null  object
 9   repair         304076 non-null  object
 10  postalcode     304076 non-null  int64 
 11  year_created   304076 non-null  int64 
 12  month_created  304076 non-null  int64 
 13  day_created    304076 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 34.8+ MB


Проверим наличие дубликатов, которые могли появиться после предобработки данных

In [28]:
print('Количество дубликатов:', sum(df.duplicated()))

Количество дубликатов: 414


Eдалим их, и заодно отбросим день подачи объявления и индекс пользователей

In [29]:
# Удалим дубликаты
df.drop_duplicates(inplace = True)

# Отбросим столбцы
df.drop(['postalcode', 'day_created'], axis = 1, inplace = True)

Посмотрим итоговый результат получившейся таблицы

In [30]:
df.head()

,price,body,year_reg,gear,power,model,mileage,fuel,brand,repair,year_created,month_created
0,480,sedan,1993,manual,101,golf,150000,petrol,volkswagen,unknown,2016,3
2,9800,suv,2004,auto,163,grand,125000,petrol,jeep,unknown,2016,3
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,2016,3
4,3600,small,2008,manual,69,fabia,90000,petrol,skoda,no,2016,3
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,2016,4


<a id='section2.2'></a>
### Подготовка выборок

Выделим признаки, целевой признак. Разделим выборку на обучающую и тестовую

In [31]:
# введём константу для random_state
RANDOM_STATE = 12345

# определение признаков 
target = df[['price']]
features = df.drop('price', axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, target, train_size = 0.9)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((273295, 11), (30367, 11), (273295, 1), (30367, 1))

In [32]:
quant_columns = ['mileage', 'power', 'year_reg']
cat_columns = ['body', 'gear', 'model', 'fuel', 'brand', 'repair']

<a id='section2.3'></a>
### Кодирование и масштабирование

Применим OrdinalEncoder к колонкам с категориальными признаками

In [33]:
ohe_encoder = OneHotEncoder(dtype = np.bool, sparse = False, handle_unknown = 'ignore')
x_train_ohe = ohe_encoder.fit_transform(x_train[cat_columns])
x_test_ohe = ohe_encoder.transform(x_test[cat_columns])
x_train_ohe.shape, x_test_ohe.shape

((273295, 307), (30367, 307))

In [34]:
ordinal_encoder = OrdinalEncoder(dtype = np.int16)
ordinal_encoder.fit(features[cat_columns])
x_train_ord = ordinal_encoder.transform(x_train[cat_columns])
x_test_ord = ordinal_encoder.transform(x_test[cat_columns])
x_train_ord.shape, x_test_ord.shape

((273295, 6), (30367, 6))

In [35]:
scaler = StandardScaler()
x_train_scal = scaler.fit_transform(x_train[quant_columns])
x_test_scal = scaler.transform(x_test[quant_columns])
x_train_scal.shape, x_test_scal.shape

((273295, 3), (30367, 3))

**Вывод:**
- Почищены выбросы в данных.
- Удалили признаки лишние признаки
- Пропуски находились только в категориальных признаках и были заполнены значением 'unknown'.
- Разделены данные на Train/Test как 80/20
- Признаки разделены на количественные (x_train_scal, x_test_scal) и категориальные(x_train_ohe, x_test_ohe)(x_train_ord, x_test_ord).
- Количественные стандартизировали при помощи StandardScaler
- Для категориальных признаков использовался OneHotEncoder и OrdinalEncoder

<a id='section3'></a>
## Обучение моделей

Для обучения будем использовать следующие модели DummyRegressor, LinearRegression, CatBoostRegressor, LGBMRegressor

<a id='section3.1'></a>
### DummyRegressor

In [36]:
%%time
DM = DummyRegressor()
DM.fit(x_train, y_train)

CPU times: user 3.01 ms, sys: 0 ns, total: 3.01 ms
Wall time: 2.21 ms


DummyRegressor()

In [55]:
%%time
predict = DM.predict(x_train)
DM_RMSE = mse(y_train, predict)
print(f'RMSE:{DM_RMSE**0.5:.2f}')

RMSE:4557.46
CPU times: user 5.63 ms, sys: 46 µs, total: 5.68 ms
Wall time: 4.69 ms


<a id='section3.2'></a>
### LinearRegression

Для LinearRegression лучше подойдут OneHot features

In [38]:
x_train_scal_ohe = np.concatenate((x_train_ohe, x_train_scal), axis = -1)

In [39]:
%%time
LM = LinearRegression()
LM.fit(x_train_scal_ohe, y_train)

CPU times: user 21.9 s, sys: 21.1 s, total: 43 s
Wall time: 43.2 s


LinearRegression()

In [53]:
%%time
predict = LM.predict(x_train_scal_ohe)
LM_RMSE = mean_squared_error(y_train, predict)
print(f'RMSE:{LM_RMSE**0.5:.2f}')

RMSE:2885.51
CPU times: user 172 ms, sys: 66.6 ms, total: 238 ms
Wall time: 180 ms


<a id='section3.3'></a>
### CatBoostRegressor

Для CatBoost и Lightgb понядобятся Ordinat features

In [41]:
x_train_ord_scale = pd.concat((pd.DataFrame(x_train_ord, columns=cat_columns),
                               pd.DataFrame(x_train_scal, columns=quant_columns)), axis=1)
train_pool = Pool(data = x_train_ord_scale, cat_features = cat_columns)

In [42]:
%%time
train_pool = Pool(data = x_train_ord_scale, label = y_train, cat_features = cat_columns)
params = {"iterations": 400,
          "learning_rate": 1,
          "max_depth": None,
          "loss_function": "RMSE",
          "random_state": RANDOM_STATE,
          "bootstrap_type": None,
          "min_data_in_leaf": 1,
          "boosting_type": 'Plain',
          "verbose": False}
bootstrap_type = ['MVS', 'Bernoulli']
max_depth = [2, 3, 4]        


CPU times: user 244 ms, sys: 7.65 ms, total: 252 ms
Wall time: 257 ms


In [43]:
%%time
best_score = 10000
for trap_type in bootstrap_type:
    params['bootstrap_type'] = trap_type
    for m_depth in max_depth:
        params['max_depth'] = m_depth
        score = cv(pool = train_pool, params = params, fold_count = 2, plot = False).iloc[-1]['test-RMSE-mean']
        if score < best_score:
            best_score = score
            best_param = (trap_type, m_depth)
            
boots_type = best_param[0]
max_depth = best_param[1]
CBR = CatBoostRegressor(iterations = 400, learning_rate = 1, max_depth = max_depth, bootstrap_type = boots_type,
                       random_state=42, boosting_type = 'Plain', verbose = False)
CBR_fit = CBR.fit(train_pool);

Training on fold [0/2]

bestTest = 1770.012782
bestIteration = 399

Training on fold [1/2]

bestTest = 1771.667855
bestIteration = 399

Training on fold [0/2]

bestTest = 1729.011545
bestIteration = 399

Training on fold [1/2]

bestTest = 1714.794212
bestIteration = 399

Training on fold [0/2]

bestTest = 1705.972694
bestIteration = 397

Training on fold [1/2]

bestTest = 1702.799707
bestIteration = 399

Training on fold [0/2]

bestTest = 1787.025638
bestIteration = 399

Training on fold [1/2]

bestTest = 1787.759485
bestIteration = 395

Training on fold [0/2]

bestTest = 1742.641886
bestIteration = 395

Training on fold [1/2]

bestTest = 1739.821412
bestIteration = 392

Training on fold [0/2]

bestTest = 1718.704438
bestIteration = 375

Training on fold [1/2]

bestTest = 1708.073615
bestIteration = 399

CPU times: user 6min, sys: 6.76 s, total: 6min 6s
Wall time: 6min 19s


In [44]:
%%time
predict = CBR.predict(train_pool)
CBR_RMSE = mse(y_train, predict)
print(f'CatBoostRegressor RMSE: {CBR_RMSE**0.5:.2f}')

CatBoostRegressor RMSE: 1607.90
CPU times: user 621 ms, sys: 0 ns, total: 621 ms
Wall time: 619 ms


<a id='section3.4'></a>
### LGBMRegressor

In [45]:
%%time
lgbm_train_data = lightgbm.Dataset(x_train_ord_scale, label = y_train, categorical_feature = cat_columns, free_raw_data = False)
param = {'learning_rate': 1,
         'metric': 'rmse',
         'stratify': False,
         'seed': 42,
         'objective': 'regression',
         'verbosity': -1,
        }
booster_method = ['gbdt', 'gross']
max_depth = [2, 3, 4]

CPU times: user 22 µs, sys: 1 µs, total: 23 µs
Wall time: 26 µs


In [46]:
%%time
best_score = 10000
for boost in booster_method:
    params['boosting'] = boost
    for m_depth in max_depth:
        params['max_depth'] = m_depth
        score = lightgbm.cv(param, lgbm_train_data, nfold = 3, categorical_feature = cat_columns)
        score = pd.DataFrame(score).iloc[-1]['rmse-mean']
        if score < best_score:
            best_score = score
            best_param = (boost, m_depth)

param['max_depth'] = best_param[1]
param['boosting'] = best_param[0]
LGBM = lightgbm.train(param, lgbm_train_data, categorical_feature=cat_columns, num_boost_round=400)

[LightGBM] [Warning] Unknown parameter: stratify
CPU times: user 16min 55s, sys: 8.45 s, total: 17min 4s
Wall time: 17min 11s


In [47]:
%%time
predict = LGBM.predict(x_train_ord_scale)
LGBM_RMSE = mse(y_train, predict)
print(f'LGBMRegressor RMSE: {LGBM_RMSE**0.5:.2f}')

LGBMRegressor RMSE: 1659.51
CPU times: user 3.81 s, sys: 0 ns, total: 3.81 s
Wall time: 3.81 s


**Вывод:**
Обучены 4 модели которые имеют следующие результаты:
- DummyRegressor: время обучения 2.21 ms и точность RMSE = 4557.46
- LinearRegression: время обучения 43.2 s, точность RMSE = 2885.51
- CatBoostRegressor: время обучения 6 min 19 s, и точность RMSE = 1607.90
- LGBMRegressor: время обучения 17 min 11 s, и точность RMSE = 1659.51

<a id='section4'></a>
## Анализ моделей

Выведем результаты обучения и предсказания моделей в общую таблицу для наглядного сравнения

In [61]:
index = ['DummyRegressor',
         'LinearRegression',
         'CatBoostRegressor',
         ' LGBMRegressor']
data = {'RMSE':[DM_RMSE**0.5,
                LM_RMSE**0.5,
                 CBR_RMSE**0.5,
                  LGBM_RMSE**0.5],
        'Время обучения модели':['2.21 ms',
                                     '43.2 s',
                                     '6 min 19 s',
                                     '17 min 11 s'],
        'Время предсказания модели':['3.44 ms',
                                          '256 ms',
                                          '619 ms',
                                          '3.81 s']}

scores_data = pd.DataFrame(data=data, index=index)

scores_data

,RMSE,Время обучения модели,Время предсказания модели
DummyRegressor,4557.458142,2.21 ms,3.44 ms
LinearRegression,2885.508165,43.2 s,256 ms
CatBoostRegressor,1607.904581,6 min 19 s,619 ms
LGBMRegressor,1659.514324,17 min 11 s,3.81 s


**Вывод:**

LGBMRegressor и CatBoostRegressor показали лучший RMSE. LGBMRegressor имеет значения RMSE = 1659.51 и время обучения 17 min 11 s, CatBoostRegressor имеет лучшие показатели, RMSE = 1607.90 и время обучения 6 min 19 s.

<a id='section4.1'></a>
### Тестирование лучших моделей

Протестируем лучшие модели на тестовой выборке

In [58]:
x_test_ord_scale = pd.concat((pd.DataFrame(x_test_ord, columns = cat_columns),
                               pd.DataFrame(x_test_scal, columns = quant_columns)), axis=1)
test_pool = Pool(data = x_test_ord_scale, cat_features = cat_columns)

In [59]:
%%time
predict = CBR.predict(test_pool)
print(f'CatBoostRegressor RMSE: {mse(y_test, predict)**0.5:.2f}')

CatBoostRegressor RMSE: 1708.37
CPU times: user 79.9 ms, sys: 153 µs, total: 80.1 ms
Wall time: 78.9 ms


In [60]:
%%time
predict = LGBM.predict(x_test_ord_scale)
print(f'LGBMRegressor RMSE: {mse(y_test, predict)**0.5:.2f}')

LGBMRegressor RMSE: 1751.41
CPU times: user 438 ms, sys: 0 ns, total: 438 ms
Wall time: 379 ms


**Вывод:**

CatBoostRegressor показал так же лучший RMSE: 1722.02, и время расчета 78.9 ms в то время как LGBMRegressor показал RMSE 1751.41 и время - 379 ms.

<a id='section5'></a>
## Общий вывод

- Почищены выбросы. 
- Удалены признаки, которые не были необходимыми в обучении моделей.
- Пропуски заполнены по возможности или заменены "значением-заглушкой".
- Разделены данные на Train/Test как 80/20
- Признаки разделены на количественные (x_train_scal, x_test_scal) и категориальные(x_train_ohe, x_test_ohe)(x_train_ord, x_test_ord).
- Количественные стандартизировали при помощи StandardScaler
- Для категориальных признаков использовался OneHotEncoder и OrdinalEncoder

Обучены 4 модели:

- DummyRegressor: время обучения 2.21 ms и точность RMSE = 4557.46
- LinearRegression: время обучения 43.2 s, точность RMSE = 2885.51
- CatBoostRegressor: время обучения 6 min 19 s, и точность RMSE = 1607.90
- LGBMRegressor: время обучения 17 min 11 s, и точность RMSE = 1659.51

LGBMRegressor и CatBoostRegressor показали лучший RMSE. Результат тестирования показал, что точность у моделей примерно одинакова. но CatBoostRegressor всё же лидирует, как по скорости обучения и предсказания, так и по значению RMSE.
Окончательной моделью лучше выбрать CatBoostRegressor